# <center> Machine Learning con PySpark </center>

#### Autor: Rodrigo Accurso

## Introducción

El objetivo de esta práctica es la realización de un proceso completo de aprendizaje automático. Está desarrollada con PySpark, utilizando las librerias ML para DataFrames.

El dataset se puede descargar directamente de Kaggle con este link:

https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset

La variable objetivo es categórica y posee dos valores (0,1), por lo cual el modelo a entrenar es de tipo clasificación binaria.

He creado 5 pipelines para comparar los resultados obtenidos aplicando técnicas de selección y extracción de variables, o ninguna de las dos:
1. Sin métodos de selección o extracción de variables (Base)
2. Selección de variables con Chi-cuadrado y ranking con umbral p-valor <= 0.05
3. Selección de variables con Chi-cuadrado y ranking con umbral p-valor <= 0.01
4. Extracción de variables con PCA y k = 5
5. Extracción de variables con PCA y k = 10

Para cada uno de estos pipelines, he generado modelos de clasificación con los siguientes algoritmos:
* Bayes Ingénuo
* Support Vector Machienes
* Random Forest
* Gradient Boosting
* Redes Neuronales

El procedimiento de evaluación consiste en la validación cruzada con 5 carpetas. La métrica principal es el área bajo la curva ROC, pero también he calculado la exactitud del modelo final.

El tuning de los hiper-parámetros se basa en el método del Grid Search, o sea asigno una serie de valores a cada uno y pruebo todas las combinaciones posibles para obtener el mejor resultado.

## Contenido
1. [Análisis exploratorio](#analisis-exploratorio)
2. [Ingeniería de variables](#ingenieria-de-variables)
3. [Bayes-Ingénuo](#bayes-ingenuo)
4. [Support Vector Machines](#svm)
5. [Random Forest](#random-forest)
6. [Gradient Boosting Tree](#gbt)
7. [Multilayer Perceptron](#perceptron)

## 1. Análisis exploratorio 
<a class="anchor" id="analisis-exploratorio"></a>

#### 1.1 Import de las librerias y lectura del dataset

In [1]:
import pyspark
from pyspark.sql.functions import isnan, when, count, countDistinct
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes, LinearSVC, RandomForestClassifier, GBTClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import ChiSqSelector, PCA
import numpy as np

sc = pyspark.SparkContext(appName="PracticaFinal")
sql = pyspark.SQLContext(sc)

In [2]:
df = sql.read.csv('HR.Employee.Attrition.csv', sep=",", inferSchema=True, header=True)
print(df.count())
print(len(df.columns))
print(df.take(2))

1470
35
[Row(Age=41, Attrition='Yes', BusinessTravel='Travel_Rarely', DailyRate=1102, Department='Sales', DistanceFromHome=1, Education=2, EducationField='Life Sciences', EmployeeCount=1, EmployeeNumber=1, EnvironmentSatisfaction=2, Gender='Female', HourlyRate=94, JobInvolvement=3, JobLevel=2, JobRole='Sales Executive', JobSatisfaction=4, MaritalStatus='Single', MonthlyIncome=5993, MonthlyRate=19479, NumCompaniesWorked=8, Over18='Y', OverTime='Yes', PercentSalaryHike=11, PerformanceRating=3, RelationshipSatisfaction=1, StandardHours=80, StockOptionLevel=0, TotalWorkingYears=8, TrainingTimesLastYear=0, WorkLifeBalance=1, YearsAtCompany=6, YearsInCurrentRole=4, YearsSinceLastPromotion=0, YearsWithCurrManager=5), Row(Age=49, Attrition='No', BusinessTravel='Travel_Frequently', DailyRate=279, Department='Research & Development', DistanceFromHome=8, Education=1, EducationField='Life Sciences', EmployeeCount=1, EmployeeNumber=2, EnvironmentSatisfaction=3, Gender='Male', HourlyRate=61, JobInvo

#### 1.2 Obtengo los tipos de variables

In [106]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- DailyRate: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- EmployeeCount: integer (nullable = true)
 |-- EmployeeNumber: integer (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- HourlyRate: integer (nullable = true)
 |-- JobInvolvement: integer (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- MonthlyRate: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- Over18: string (nullable = true)
 |-- OverTime: string 

#### 1.3 Verifico la presencia de NAs

In [123]:
df_na = df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).collect()
print(df_na)

[Row(Age=0, Attrition=0, BusinessTravel=0, DailyRate=0, Department=0, DistanceFromHome=0, Education=0, EducationField=0, EmployeeCount=0, EmployeeNumber=0, EnvironmentSatisfaction=0, Gender=0, HourlyRate=0, JobInvolvement=0, JobLevel=0, JobRole=0, JobSatisfaction=0, MaritalStatus=0, MonthlyIncome=0, MonthlyRate=0, NumCompaniesWorked=0, Over18=0, OverTime=0, PercentSalaryHike=0, PerformanceRating=0, RelationshipSatisfaction=0, StandardHours=0, StockOptionLevel=0, TotalWorkingYears=0, TrainingTimesLastYear=0, WorkLifeBalance=0, YearsAtCompany=0, YearsInCurrentRole=0, YearsSinceLastPromotion=0, YearsWithCurrManager=0)]


## 2. Ingeniería de variables
<a class="anchor" id="ingenieria-de-variables"></a>

#### 2.1 Elimino las columnas con valores diferentes en todas las filas

In [4]:
df_unique = df.select([when(countDistinct(column) == df.count(), 'T').otherwise('F').alias(column) for column in df.columns]) \
                .collect()
print(df_unique)

[Row(Age='F', Attrition='F', BusinessTravel='F', DailyRate='F', Department='F', DistanceFromHome='F', Education='F', EducationField='F', EmployeeCount='F', EmployeeNumber='T', EnvironmentSatisfaction='F', Gender='F', HourlyRate='F', JobInvolvement='F', JobLevel='F', JobRole='F', JobSatisfaction='F', MaritalStatus='F', MonthlyIncome='F', MonthlyRate='F', NumCompaniesWorked='F', Over18='F', OverTime='F', PercentSalaryHike='F', PerformanceRating='F', RelationshipSatisfaction='F', StandardHours='F', StockOptionLevel='F', TotalWorkingYears='F', TrainingTimesLastYear='F', WorkLifeBalance='F', YearsAtCompany='F', YearsInCurrentRole='F', YearsSinceLastPromotion='F', YearsWithCurrManager='F')]


In [3]:
df = df.drop('EmployeeNumber')

#### 2.2 Elimino las columnas con valor igual en todas las filas

In [54]:
df_same = df.select([when(countDistinct(column) == 1, 'T').otherwise('F').alias(column) for column in df.columns]) \
                .collect()
print(df_same)

[Row(Age='F', Attrition='F', BusinessTravel='F', DailyRate='F', Department='F', DistanceFromHome='F', Education='F', EducationField='F', EmployeeCount='T', EnvironmentSatisfaction='F', Gender='F', HourlyRate='F', JobInvolvement='F', JobLevel='F', JobRole='F', JobSatisfaction='F', MaritalStatus='F', MonthlyIncome='F', MonthlyRate='F', NumCompaniesWorked='F', Over18='T', OverTime='F', PercentSalaryHike='F', PerformanceRating='F', RelationshipSatisfaction='F', StandardHours='T', StockOptionLevel='F', TotalWorkingYears='F', TrainingTimesLastYear='F', WorkLifeBalance='F', YearsAtCompany='F', YearsInCurrentRole='F', YearsSinceLastPromotion='F', YearsWithCurrManager='F')]


In [4]:
df = df.drop('EmployeeCount')
df = df.drop('Over18')
df = df.drop('StandardHours')

#### 2.3 Convierto las variables alfanumericas en numericas

In [5]:
main_stages = []
string_cols = [x[0] for x in df.dtypes if (x[1] == 'string') & (x[0] != 'Attrition')]
string_cols

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [6]:
numeric_cols = [x[0] for x in df.dtypes if x[1] != 'string']
numeric_cols

['Age',
 'DailyRate',
 'DistanceFromHome',
 'Education',
 'EnvironmentSatisfaction',
 'HourlyRate',
 'JobInvolvement',
 'JobLevel',
 'JobSatisfaction',
 'MonthlyIncome',
 'MonthlyRate',
 'NumCompaniesWorked',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

In [7]:
for col in string_cols:
    indexer = StringIndexer(inputCol = col, outputCol = col + 'Index')
    main_stages += [indexer]

In [8]:
# Transformo la variable target Attrition separadamente porque no debe estar en el pipeline
indexer = StringIndexer(inputCol = 'Attrition', outputCol = 'label')
indexer = indexer.fit(df)
df = indexer.transform(df)

#### 2.4 Aplico el One Hot Encoding en las variables categoricas

In [9]:
cat_cols = ['Department', 'EducationField','JobRole','MaritalStatus']

In [10]:
for col in cat_cols:
    encoder = OneHotEncoderEstimator(inputCols = [col + 'Index'], outputCols = [col + 'Vec'])
    main_stages += [encoder]

#### 2.5 Genero el vector necesario para entrenar los modelos de ML

In [11]:
# Variables numericas
assemblerInputs = numeric_cols
# Variables alfanumericas a las que no aplique el one hot encoding
assemblerInputs = assemblerInputs + [col + 'Index' for col in (set(string_cols) - set(cat_cols))]
# Variables alfanumericas a las que aplique el one hot encoding
assemblerInputs = assemblerInputs + [col + 'Vec' for col in cat_cols]

In [12]:
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol='features')
main_stages += [assembler]

#### 2.6 Normalizacion de las variables

In [13]:
scaler = MinMaxScaler(inputCol='features', outputCol='scaledFeatures')
main_stages += [scaler]

#### 2.7 Selección de variables por Chi-cuadrado

Selección con p-valor <= 0.05

In [14]:
chisq_selector_05 = ChiSqSelector(fpr=0.05, selectorType='fpr',featuresCol='scaledFeatures',
                         outputCol='selectedFeatures', labelCol='label')
chisq_stages_05 = main_stages[:]
chisq_stages_05 += [chisq_selector_05]

Selección con p-valor <= 0.01

In [15]:
chisq_selector_01 = ChiSqSelector(fpr=0.01, selectorType='fpr',featuresCol='scaledFeatures',
                         outputCol='selectedFeatures', labelCol='label')
chisq_stages_01 = main_stages[:]
chisq_stages_01 += [chisq_selector_01]

#### 2.8 Extracción de variables con PCA

Extracción con k = 5

In [16]:
pca_5 = PCA(k=5, inputCol='scaledFeatures', outputCol='pcaFeatures')
pca_stages_5 = main_stages[:]
pca_stages_5 += [pca_5]

Extracción con k = 10

In [17]:
pca_10 = PCA(k=10, inputCol='scaledFeatures', outputCol='pcaFeatures')
pca_stages_10 = main_stages[:]
pca_stages_10 += [pca_10]

## 3. Bayes Ingénuo
<a class="anchor" id="bayes-ingenuo"></a>

#### 3.1 Creo diccionarios con la información necesaria para ejecutar todos los casos por cada aloritmo

In [146]:
all_stages = {'BASE': main_stages,
              'CHI-CUADRADO-05': chisq_stages_05,
              'CHI-CUADRADO-01': chisq_stages_01}

feature_field = {'BASE': 'scaledFeatures',
                 'CHI-CUADRADO-05': 'selectedFeatures',
                 'CHI-CUADRADO-01': 'selectedFeatures'}

Nota: No puedo utilizar PCA porque el Bayes Ingénuo no acepta números negativos

#### 3.2 Creo una función que evalúa el algoritmo con los hiper-parámetros en entrada

In [147]:
def evualua_modelo(input_smoothing):
    evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
    for sel_stages in all_stages:
        
        print('CASO ' + sel_stages)
        print('------------------------')
        
        # Creo el algoritmo de clasificación
        nb = NaiveBayes(featuresCol=feature_field.get(sel_stages), labelCol='label')    
        print('Features: ' + feature_field.get(sel_stages))
        
        # Construyo el pipeline completo
        nb_stages = all_stages.get(sel_stages)[:]
        nb_stages += [nb]
        pipeline = Pipeline(stages=nb_stages)

        # Creo el grid de hiper-parámetros
        paramGrid = (ParamGridBuilder()
                     .addGrid(nb.smoothing, input_smoothing.get(sel_stages))
                     .addGrid(nb.modelType, ['multinomial'])
                     .build())  

        # Ejecuto la validación cruzada con los hiperparámetros seleccionados
        cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, 
                            evaluator=evaluator, numFolds=5)
        pipelineModel = cv.fit(df)    
        
        # Muestro los resultados
        print('Hiper-parámetros óptimos:')
        print('smoothing = ' + str(pipelineModel.bestModel.stages[-1]._java_obj.getSmoothing()))
        print('ROC-AUC = ' + str(np.mean(pipelineModel.avgMetrics)))
        print('')    

#### 3.3 Coarse-tuning de hiper-parámetros 

In [148]:
param_smoothing = {'BASE': [0., 0.5, 1.0],
                   'CHI-CUADRADO-05': [0., 0.5, 1.0],
                   'CHI-CUADRADO-01':  [0., 0.5, 1.0]}                   
    
evualua_modelo(param_smoothing)

CASO BASE
------------------------
Features: scaledFeatures
Hiper-parámetros óptimos:
smoothing = 0.0
ROC-AUC = 0.6305925693344068

CASO CHI-CUADRADO-05
------------------------
Features: selectedFeatures
Hiper-parámetros óptimos:
smoothing = 0.0
ROC-AUC = 0.6314127561352433

CASO CHI-CUADRADO-01
------------------------
Features: selectedFeatures
Hiper-parámetros óptimos:
smoothing = 0.0
ROC-AUC = 0.6334527955217494



#### 3.4 Fine-tuning de hiper-parámetros y evaluación de la ROC-AUC

In [150]:
param_smoothing = {'BASE': [0., 0.05, .1],
                   'CHI-CUADRADO-05': [0., 0.05, .1],
                   'CHI-CUADRADO-01':  [0., 0.05, .1]}                   
    
evualua_modelo(param_smoothing)

CASO BASE
------------------------
Features: scaledFeatures
Hiper-parámetros óptimos:
smoothing = 0.0
ROC-AUC = 0.6310011695464576

CASO CHI-CUADRADO-05
------------------------
Features: selectedFeatures
Hiper-parámetros óptimos:
smoothing = 0.0
ROC-AUC = 0.6320403657626575

CASO CHI-CUADRADO-01
------------------------
Features: selectedFeatures
Hiper-parámetros óptimos:
smoothing = 0.0
ROC-AUC = 0.6338857821851124



#### 3.5 Calculo la exactitud del mejor modelo

In [291]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
nb = NaiveBayes(featuresCol='selectedFeatures', labelCol='label')

nb_stages = chisq_stages_01[:]
nb_stages += [nb]
pipeline = Pipeline(stages=nb_stages)

paramGrid = (ParamGridBuilder()
             .addGrid(nb.smoothing, [0.])
             .build())  

cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, 
                    evaluator=evaluator, numFolds=5)
model = cv.fit(df)  
print('Accuracy = ' + str(np.mean(model.avgMetrics)))

Accuracy = 0.8435268000387981


#### 3.6 Resultados

El único hiper-parámetro configurable es el Smoothing, que es la suavización de probabilidades a través del Estimador de Laplace. El valor óptimo es 0 porque en el dataset no existe el caso en que el valor de una variable categórica no se esté en una de las dos clases.

La ejecución con selección de variables con p-valor <= 0.01 obtuvo el mejor resultado. Supongo que el motivo es la no independencia de variables que existe en el dataset, mientras que Bayes Ingénuo asume una completa independencia.

ROC-AUC = 0.6339

Accuracy = 0.8435

## 4. Support Vector Machines
<a class="anchor" id="svm"></a>

#### 4.1 Creo diccionarios con la información necesaria para ejecutar todos los casos por cada aloritmo

In [69]:
all_stages = {'BASE': main_stages,
              'CHI-CUADRADO-05': chisq_stages_05,
              'CHI-CUADRADO-01': chisq_stages_01,
              'PCA-k5': pca_stages_5,
              'PCA-k10': pca_stages_10}

feature_field = {'BASE': 'scaledFeatures',
                 'CHI-CUADRADO-05': 'selectedFeatures',
                 'CHI-CUADRADO-01': 'selectedFeatures',
                 'PCA-k5': 'pcaFeatures',
                 'PCA-k10': 'pcaFeatures'}

#### 4.2 Creo una función que evalúa el algoritmo con los hiper-parámetros en entrada

In [158]:
def evualua_modelo_SVM(input_regParam, input_maxIter):
    evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
    for sel_stages in all_stages:
        
        print('CASO ' + sel_stages)
        print('------------------------')
        
        # Creo el algoritmo de clasificación
        svc = LinearSVC(featuresCol=feature_field.get(sel_stages), labelCol='label')    
        print('Features: ' + feature_field.get(sel_stages))
        
        # Construyo el pipeline completo
        svc_stages = all_stages.get(sel_stages)[:]
        svc_stages += [svc]
        pipeline = Pipeline(stages=svc_stages)

        # Creo el grid de hiper-parámetros
        paramGrid = (ParamGridBuilder()
                     .addGrid(svc.regParam, input_regParam.get(sel_stages))
                     .addGrid(svc.maxIter, input_maxIter.get(sel_stages))                  
                     .build())  

        # Ejecuto la validación cruzada con los hiperparámetros seleccionados
        cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, 
                            evaluator=evaluator, numFolds=5)
        pipelineModel = cv.fit(df)    
        
        # Muestro los resultados
        print('Hiper-parámetros óptimos:')
        print('regParam = ' + str(pipelineModel.bestModel.stages[-1]._java_obj.getRegParam()))
        print('maxIter = ' + str(pipelineModel.bestModel.stages[-1]._java_obj.getMaxIter()))
        print('ROC-AUC = ' + str(np.mean(pipelineModel.avgMetrics)))
        print('')    

#### 4.3 Coarse-tuning de hiper-parámetros 

In [169]:
param_regParam = {'BASE': [0., 0.5, 1.0],
                  'CHI-CUADRADO-05': [0., 0.5, 1.0],
                  'CHI-CUADRADO-01':  [0., 0.5, 1.0],
                  'PCA-k5': [0., 0.5, 1.0],
                  'PCA-k10': [0., 0.5, 1.0]}

param_maxIter = {'BASE': [10, 50, 100],
                 'CHI-CUADRADO-05': [10, 50, 100],
                 'CHI-CUADRADO-01': [10, 50, 100],
                 'PCA-k5': [10, 50, 100],
                 'PCA-k10': [10, 50, 100]}
    
evualua_modelo_SVM(param_regParam, param_maxIter)

CASO BASE
------------------------
Features: scaledFeatures
Hiper-parámetros óptimos:
regParam = 0.5
maxIter = 100
ROC-AUC = 0.8143945767212432

CASO CHI-CUADRADO-05
------------------------
Features: selectedFeatures
Hiper-parámetros óptimos:
regParam = 1.0
maxIter = 50
ROC-AUC = 0.8002146337229715

CASO CHI-CUADRADO-01
------------------------
Features: selectedFeatures
Hiper-parámetros óptimos:
regParam = 0.5
maxIter = 100
ROC-AUC = 0.798068156553817

CASO PCA-k5
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
regParam = 0.0
maxIter = 10
ROC-AUC = 0.654768298041164

CASO PCA-k10
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
regParam = 0.0
maxIter = 10
ROC-AUC = 0.7626341477018037



#### 4.4 Fine-tuning de hiper-parámetros y evaluación de la ROC-AUC

In [172]:
param_regParam = {'BASE': [0.4, 0.5, 0.6],
                  'CHI-CUADRADO-05': [0.8, 0.9, 1.0],
                  'CHI-CUADRADO-01':  [0.4, 0.5, 0.6],
                  'PCA-k5': [0., 0.05, 0.1],
                  'PCA-k10': [0., 0.05, 0.1]}

param_maxIter = {'BASE': [80, 100, 120],
                 'CHI-CUADRADO-05': [30, 50, 70],
                 'CHI-CUADRADO-01': [80, 100, 120],
                 'PCA-k5': [10, 20, 30],
                 'PCA-k10': [10, 20, 30]}
    
evualua_modelo_SVM(param_regParam, param_maxIter)

CASO BASE
------------------------
Features: scaledFeatures
Hiper-parámetros óptimos:
regParam = 0.6
maxIter = 100
ROC-AUC = 0.8293923941436913

CASO CHI-CUADRADO-05
------------------------
Features: selectedFeatures
Hiper-parámetros óptimos:
regParam = 1.0
maxIter = 50
ROC-AUC = 0.8022375679279371

CASO CHI-CUADRADO-01
------------------------
Features: selectedFeatures
Hiper-parámetros óptimos:
regParam = 0.5
maxIter = 80
ROC-AUC = 0.8046145747715915

CASO PCA-k5
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
regParam = 0.0
maxIter = 10
ROC-AUC = 0.6437187105127199

CASO PCA-k10
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
regParam = 0.1
maxIter = 10
ROC-AUC = 0.7625266621007054



#### 4.5 Calculo la exactitud del mejor modelo

In [277]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
svc = LinearSVC(featuresCol='scaledFeatures', labelCol='label')

svc_stages = main_stages[:]
svc_stages += [svc]
pipeline = Pipeline(stages=svc_stages)

paramGrid = (ParamGridBuilder()
             .addGrid(svc.regParam, [0.6])
             .addGrid(svc.maxIter, [100])  
             .build())  

cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, 
                    evaluator=evaluator, numFolds=5)
model = cv.fit(df)  
print('Accuracy = ' + str(np.mean(model.avgMetrics)))

Accuracy = 0.8385337330124533


#### 4.6 Resultados

El parámetro de regularización en SVM sirve a penalizar las clasificaciones erradas. Más alto su valor, mayor será el precio a pagar en la función de evaluación. El valor óptimo encontrado es 0.01.
El parámetro maxIter, en cambio, limita el número de iteraciones para el entrenamiento. El valor óptimo encontrado es 120.

Entre los 5 escenarios probados, obtuve mejor resultado sin aplicar selección o extracción de variables.

ROC-AUC = 0.8294

Accuracy = 0.8385

## 5. Random Forest
<a class="anchor" id="random-forest"></a>

#### 5.1 Creo diccionarios con la información necesaria para ejecutar todos los casos por cada aloritmo

In [142]:
all_stages = {'BASE': main_stages,
              'PCA-k5': pca_stages_5,
              'PCA-k10': pca_stages_10}

feature_field = {'BASE': 'scaledFeatures',
                 'PCA-k5': 'pcaFeatures',
                 'PCA-k10': 'pcaFeatures'}

Nota: Random Forest es un algoritmo basado en árboles de decisión, los cuales realizan internamente la selección de variables. Por este motivo, excluyo los dos pipelines con selección de variables por chi-cuadrado.

#### 5.2 Creo una función que evalúa el algoritmo con los hiper-parámetros en entrada

In [143]:
def evualua_modelo_RF(input_numTrees, input_maxDepth):
    evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
    for sel_stages in all_stages:
        
        print('CASO ' + sel_stages)
        print('------------------------')
        
        # Creo el algoritmo de clasificación
        rf = RandomForestClassifier(featuresCol=feature_field.get(sel_stages), labelCol='label')
        print('Features: ' + feature_field.get(sel_stages))
        
        # Construyo el pipeline completo
        rf_stages = all_stages.get(sel_stages)[:]
        rf_stages += [rf]
        pipeline = Pipeline(stages=rf_stages)

        # Creo el grid de hiper-parámetros
        paramGrid = (ParamGridBuilder()
                     .addGrid(rf.numTrees, input_numTrees.get(sel_stages))
                     .addGrid(rf.maxDepth, input_maxDepth.get(sel_stages))                  
                     .build())  

        # Ejecuto la validación cruzada con los hiperparámetros seleccionados
        cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, 
                            evaluator=evaluator, numFolds=5)
        pipelineModel = cv.fit(df)    
        
        # Muestro los resultados
        print('Hiper-parámetros óptimos:')
        print('numTrees = ' + str(pipelineModel.bestModel.stages[-1]._java_obj.getNumTrees()))
        print('maxDepth = ' + str(pipelineModel.bestModel.stages[-1]._java_obj.getMaxDepth()))
        print('ROC-AUC = ' + str(np.mean(pipelineModel.avgMetrics)))
        print('')    

#### 5.3 Coarse-tuning de hiper-parámetros 

In [144]:
param_numTrees = {'BASE': [50, 200, 300],
                  'PCA-k5': [50, 200, 300],
                  'PCA-k10': [50, 200, 300]}

param_maxDepth = {'BASE': [5, 10, 15],
                 'PCA-k5': [5, 10, 15],
                 'PCA-k10': [5, 10, 15]}
    
evualua_modelo_RF(param_numTrees, param_maxDepth)

CASO BASE
------------------------
Features: scaledFeatures
Hiper-parámetros óptimos:
numTrees = 200
maxDepth = 15
ROC-AUC = 0.7977902282992116

CASO PCA-k5
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
numTrees = 200
maxDepth = 5
ROC-AUC = 0.7088523134138865

CASO PCA-k10
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
numTrees = 300
maxDepth = 10
ROC-AUC = 0.7593208604087995



#### 5.4 Fine-tuning de hiper-parámetros y evaluación de la ROC-AUC

In [149]:
param_numTrees = {'BASE': [180, 200, 220],
                  'PCA-k5': [180, 200, 220],
                  'PCA-k10': [280, 300, 320]}

param_maxDepth = {'BASE': [13, 15, 17],
                 'PCA-k5': [3, 5, 7],
                 'PCA-k10': [8, 10, 12]}
    
evualua_modelo_RF(param_numTrees, param_maxDepth)

CASO BASE
------------------------
Features: scaledFeatures
Hiper-parámetros óptimos:
numTrees = 220
maxDepth = 15
ROC-AUC = 0.8027401665016947

CASO PCA-k5
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
numTrees = 200
maxDepth = 5
ROC-AUC = 0.7176632423658518

CASO PCA-k10
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
numTrees = 320
maxDepth = 8
ROC-AUC = 0.7665432457275998



#### 5.5 Calculo la exactitud del mejor modelo

In [278]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
rf = RandomForestClassifier(featuresCol='scaledFeatures', labelCol='label')

rf_stages = main_stages[:]
rf_stages += [rf]
pipeline = Pipeline(stages=rf_stages)

paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [220])
             .addGrid(rf.maxDepth, [15])     
             .build())  

cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, 
                    evaluator=evaluator, numFolds=5)
model = cv.fit(df)  
print('Accuracy = ' + str(np.mean(model.avgMetrics)))

Accuracy = 0.8572428924529123


#### 5.6 Resultados

El hiper-parámetro numTrees es el número de árboles que serán creados. Un valor alto puede provocar overfitting, mientras que uno bajo puede dar lungar al underfitting. El valor óptimo encontrado es 220 para el caso Base.

El hiper-parámetro maxDepth representa la profundidad máxima de cada árbol. También en este caso, un valor muy alto causa overfitting. El valor óptimo encontrado es 15 para el caso Base.

La mejor eficacia la obtuve con el caso sin selección ni extracción de variables (BASE). Esto tiene sentido, ya que los algoritmos basados en árboles de decisión tienen integrada la selección de las variables más predictivas.

ROC-AUC = 0.8027

Accuracy = 0.8572

## 6. Gradient Boosting Tree
<a class="anchor" id="gbt"></a>

#### 6.1 Creo diccionarios con la información necesaria para ejecutar todos los casos por cada aloritmo

In [21]:
all_stages = {'BASE': main_stages,
              'PCA-k5': pca_stages_5,
              'PCA-k10': pca_stages_10}

feature_field = {'BASE': 'scaledFeatures',
                 'PCA-k5': 'pcaFeatures',
                 'PCA-k10': 'pcaFeatures'}

Nota: Gradient Boosted Tree es un algoritmo basado en árboles de decisión, los cuales realizan internamente la selección de variables. Por este motivo, excluyo los dos pipelines con selección de variables por chi-cuadrado.

#### 6.2 Creo una función que evalúa el algoritmo con los hiper-parámetros en entrada

In [19]:
def evualua_modelo_GBT(input_maxIter, input_maxDepth):
    evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
    for sel_stages in all_stages:
        
        print('CASO ' + sel_stages)
        print('------------------------')
        
        # Creo el algoritmo de clasificación
        gbt = GBTClassifier(featuresCol=feature_field.get(sel_stages), labelCol='label')
        print('Features: ' + feature_field.get(sel_stages))
        
        # Construyo el pipeline completo
        gbt_stages = all_stages.get(sel_stages)[:]
        gbt_stages += [gbt]
        pipeline = Pipeline(stages=gbt_stages)

        # Creo el grid de hiper-parámetros
        paramGrid = (ParamGridBuilder()
                     .addGrid(gbt.maxIter, input_maxIter.get(sel_stages))
                     .addGrid(gbt.maxDepth, input_maxDepth.get(sel_stages))                  
                     .build())  

        # Ejecuto la validación cruzada con los hiperparámetros seleccionados
        cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, 
                            evaluator=evaluator, numFolds=5)
        pipelineModel = cv.fit(df)    
        
        # Muestro los resultados
        print('Hiper-parámetros óptimos:')
        print('maxIter = ' + str(pipelineModel.bestModel.stages[-1]._java_obj.getMaxIter()))
        print('maxDepth = ' + str(pipelineModel.bestModel.stages[-1]._java_obj.getMaxDepth()))
        print('ROC-AUC = ' + str(np.mean(pipelineModel.avgMetrics)))
        print('')    

#### 6.3 Coarse-tuning de hiper-parámetros 

In [22]:
param_maxIter = {'BASE': [40, 60, 80],
                 'PCA-k5': [40, 60, 80],
                 'PCA-k10': [40, 60, 80]}

param_maxDepth = {'BASE': [2, 4, 6],
                 'PCA-k5': [2, 4, 6],
                 'PCA-k10': [2, 4, 6],}
    
evualua_modelo_GBT(param_maxIter, param_maxDepth)

CASO BASE
------------------------
Features: scaledFeatures
Hiper-parámetros óptimos:
maxIter = 80
maxDepth = 2
ROC-AUC = 0.7920814451235488

CASO PCA-k5
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
maxIter = 60
maxDepth = 2
ROC-AUC = 0.6965980448249756

CASO PCA-k10
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
maxIter = 80
maxDepth = 2
ROC-AUC = 0.7289748309589227



#### 6.4 Fine-tuning de hiper-parámetros y evaluación de la ROC-AUC

In [25]:
param_maxIter = {'BASE': [120, 130, 140],
                  'PCA-k5': [60, 70, 80],
                  'PCA-k10': [80, 90, 100]}

param_maxDepth = {'BASE': [2, 3],
                 'PCA-k5': [2, 3],
                 'PCA-k10': [2, 3]}
    
evualua_modelo_GBT(param_maxIter, param_maxDepth)

CASO BASE
------------------------
Features: scaledFeatures
Hiper-parámetros óptimos:
maxIter = 140
maxDepth = 2
ROC-AUC = 0.8155666333735284

CASO PCA-k5
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
maxIter = 70
maxDepth = 3
ROC-AUC = 0.7111847107152408

CASO PCA-k10
------------------------
Features: pcaFeatures
Hiper-parámetros óptimos:
maxIter = 90
maxDepth = 2
ROC-AUC = 0.7498059022616846



#### 6.5 Calculo la exactitud del mejor modelo

In [279]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
gbt = GBTClassifier(featuresCol='scaledFeatures', labelCol='label')

gbt_stages = main_stages[:]
gbt_stages += [gbt]
pipeline = Pipeline(stages=gbt_stages)

paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxIter, [140])
             .addGrid(gbt.maxDepth, [2])     
             .build())  

cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, 
                    evaluator=evaluator, numFolds=5)
model = cv.fit(df)  
print('Accuracy = ' + str(np.mean(model.avgMetrics)))

Accuracy = 0.8686679524939078


#### 7.5 Resultados

El hiper-parámetro numIter es el número máximo de iteraciones. Un valor alto puede provocar overfitting, mientras que uno bajo puede dar lungar al underfitting. El valor óptimo encontrado es 220 para el caso Base.

El hiper-parámetro maxDepth representa la profundidad máxima de cada árbol. También en este caso, un valor muy alto causa overfitting. El valor óptimo encontrado es 15 para el caso Base.

Como con Random Forest, la mejor eficacia la obtuve sin selección ni extracción de variables, siendo basado en árboles de decisión.

ROC-AUC = 0.8155

Accuracy = 0.8686

## 7. Multilayer Perceptron
<a class="anchor" id="perceptron"></a>

#### 7.1 Creo diccionarios con la información necesaria para ejecutar todos los casos por cada aloritmo

In [267]:
all_stages = {'BASE': main_stages,
              'CHI-CUADRADO-05': chisq_stages_05,
              'CHI-CUADRADO-01': chisq_stages_01,
              'PCA-k5': pca_stages_5,
              'PCA-k10': pca_stages_10}

feature_field = {'BASE': 'scaledFeatures',
                 'CHI-CUADRADO-05': 'selectedFeatures',
                 'CHI-CUADRADO-01': 'selectedFeatures',
                 'PCA-k5': 'pcaFeatures',
                 'PCA-k10': 'pcaFeatures'}

#### 7.2 Creo una función que evalúa el algoritmo con los hiper-parámetros en entrada

In [269]:
def evualua_modelo_MP(input_maxIter, hidden_layer):
    evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
    for sel_stages in all_stages:
        
        print('CASO ' + sel_stages)
        print('------------------------')
        
        # Construyo el pipeline sin el clasificador para poder medir la cantidad de elementos
        # en el array de entrada. Debe ser igual a la cantidad de neuronas de la primera capa.
        mp_stages = all_stages.get(sel_stages)[:]
        pipeline = Pipeline(stages=mp_stages)
        pipelineModel = pipeline.fit(df)
        df_features = pipelineModel.transform(df)
        
        # Construyo las 3 capas
        num_features = len(df_features.select(feature_field.get(sel_stages)).take(1)[0][0])
        layers = [num_features, hidden_layer.get(sel_stages), 2]

        # Creo el algoritmo de clasificación
        mp = MultilayerPerceptronClassifier(featuresCol=feature_field.get(sel_stages), 
                                            labelCol='label', layers=layers,
                                            blockSize=8)
        print('Features: ' + feature_field.get(sel_stages))
        print('Layers: ' + str(layers))
        
        
        # Creo el grid de hiper-parámetros
        paramGrid = (ParamGridBuilder()
                     .addGrid(mp.maxIter, input_maxIter.get(sel_stages))
                     .build())  

        # Ejecuto la validación cruzada con los hiperparámetros seleccionados
        cv = CrossValidator(estimator=mp, estimatorParamMaps=paramGrid, 
                            evaluator=evaluator, numFolds=5)
        model = cv.fit(df_features)    
        
        # Muestro los resultados
        print('Hiper-parámetros óptimos:')
        print('maxIter = ' + str(model.bestModel._java_obj.parent().getMaxIter()))
        print('ROC-AUC = ' + str(np.mean(model.avgMetrics)))
        print('')

#### 7.3 Coarse-tuning de hiper-parámetros 

In [270]:
param_maxIter = {'BASE': [10, 20, 30],
                 'CHI-CUADRADO-05': [10, 20, 30],
                 'CHI-CUADRADO-01': [10, 20, 30],
                 'PCA-k5': [10, 20, 30],
                 'PCA-k10': [10, 20, 30]}

param_layers = {'BASE': 22,
                'CHI-CUADRADO-05': 13,
                'CHI-CUADRADO-01': 12,
                'PCA-k5': 3,
                'PCA-k10': 5}
    
evualua_modelo_MP(param_maxIter, param_layers)

CASO BASE
------------------------
Features: scaledFeatures
Layers: [43, 22, 2]
Hiper-parámetros óptimos:
maxIter = 20
ROC-AUC = 0.8075354100031692

CASO CHI-CUADRADO-05
------------------------
Features: selectedFeatures
Layers: [27, 13, 2]
Hiper-parámetros óptimos:
maxIter = 20
ROC-AUC = 0.8012664327757114

CASO CHI-CUADRADO-01
------------------------
Features: selectedFeatures
Layers: [25, 12, 2]
Hiper-parámetros óptimos:
maxIter = 20
ROC-AUC = 0.796781592603775

CASO PCA-k5
------------------------
Features: pcaFeatures
Layers: [5, 3, 2]
Hiper-parámetros óptimos:
maxIter = 20
ROC-AUC = 0.6857973215519944

CASO PCA-k10
------------------------
Features: pcaFeatures
Layers: [10, 5, 2]
Hiper-parámetros óptimos:
maxIter = 20
ROC-AUC = 0.7718899148835229



#### 7.4 Fine-tuning de hiper-parámetros y evaluación de la ROC-AUC

In [271]:
param_maxIter = {'BASE': [15, 20, 25],
                 'CHI-CUADRADO-05': [15, 20, 25],
                 'CHI-CUADRADO-01': [15, 20, 25],
                 'PCA-k5': [15, 20, 25],
                 'PCA-k10': [15, 20, 25]}

param_layers = {'BASE': 22,
                'CHI-CUADRADO-05': 13,
                'CHI-CUADRADO-01': 12,
                'PCA-k5': 3,
                'PCA-k10': 5}
    
evualua_modelo_MP(param_maxIter, param_layers)

CASO BASE
------------------------
Features: scaledFeatures
Layers: [43, 22, 2]
Hiper-parámetros óptimos:
maxIter = 20
ROC-AUC = 0.8210557007372139

CASO CHI-CUADRADO-05
------------------------
Features: selectedFeatures
Layers: [27, 13, 2]
Hiper-parámetros óptimos:
maxIter = 15
ROC-AUC = 0.8058342862861996

CASO CHI-CUADRADO-01
------------------------
Features: selectedFeatures
Layers: [25, 12, 2]
Hiper-parámetros óptimos:
maxIter = 15
ROC-AUC = 0.8040253316350424

CASO PCA-k5
------------------------
Features: pcaFeatures
Layers: [5, 3, 2]
Hiper-parámetros óptimos:
maxIter = 15
ROC-AUC = 0.695810530500201

CASO PCA-k10
------------------------
Features: pcaFeatures
Layers: [10, 5, 2]
Hiper-parámetros óptimos:
maxIter = 15
ROC-AUC = 0.774815000263342



#### 7.5 Calculo la exactitud del mejor modelo

In [285]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
mp = MultilayerPerceptronClassifier(featuresCol='scaledFeatures', 
                                    labelCol='label', layers=[43, 22, 2], blockSize=8)

mp_stages = main_stages[:]
mp_stages += [mp]
pipeline = Pipeline(stages=mp_stages)

paramGrid = (ParamGridBuilder()
             .addGrid(mp.maxIter, [20])
             .build())  

cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, 
                    evaluator=evaluator, numFolds=5)
model = cv.fit(df)  
print('Accuracy = ' + str(np.mean(model.avgMetrics)))

Accuracy = 0.879859437233464


#### 7.6 Resultados

Todas las redes neuronales poseen 3 capas y la siguiente cantidad de neuronas por cada capa:
* Primera: cantidad de features presente en el vector en entrada a la red neuronal. Depende de la ejecución de las técnicas de selección y extracción de variables.
* Segunda o hidden layer: valor comprendido entre el número de neuronas de la primera y tercera capa. Realicé varias pruebas para encontrar el valor que diera mejor resultado.
* Tercera: número de clases de la variable objetivo, o sea constante 2.

El hiper-parámetro maxIter es el número máximo de iteraciones. Un valor alto puede provocar overfitting, mientras que uno bajo puede dar lungar al underfitting. El valor óptimo encontrado es 20 para el caso Base.

Obtuve la mejor eficacia sin selección ni extracción de variables. No asombra este resultado, ya que las redes neuronales no son sensibles a la presencia variables correlacionadas.

ROC-AUC = 0.8210

Accuracy = 0.8798

## 8. Conclusión

La siguiente tabla resume los resultados obtenidos con los 5 algorítmos de clasificación. Se encuentran ordenados según la eficacia obtenida con las dos métricas.

| Algoritmo | Accuracy | ROC-AUC | Selección de variables | PCA |  
| --- | --- | --- | --- | --- |
| Multilayer Perceptron | 0.8798 | 0.8210 | No | No |
| Gradient Boosting Trees | 0.8686 | 0.8155 | No | No |
| Random Forest | 0.8572 | 0.8027 | No | No |
| Support Vector Machines | 0.8385 | 0.8294 | No | No |
| Bayes Ingénuo | 0.8435 | 0.6339 | Si | No |

La red de neuronas ofrece la mayor exactitud y casi la mejor área bajo la curva ROC. 
Una exactitud de 88% es bastante alta para un modelo predictivo y el área bajo la curva de 0.82 nos garantiza que el modelo es eficaz para varios umbrales de TPR y FPR. 

Los algoritmos basados en ensemble, como Gradient Boosting y Random Forest, estuvieron muy cerca de la red de neuronas. Demuestra que son muy eficaces, ofreciendo además resultados más claros para el ser humano.

Support Vector Machines, aún siendo un algoritmo relativamente básico, ha dado resultados excelentes para ambas métricas. En mi opinión, es el caso más sorprendente.

Bayes Ingénuo, en cambio, tiene el área bajo la curva muy baja y la exactitud alta. Obviamente, he revisado el procedimiento varias veces y el resultado sigue siendo el mismo. Supongo que tiene ese nivel de exactitud en ciertas condiciones de TPR y FPR, mientras que en el resto es bastante inferior.

In [120]:
sc.stop()